In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 设置项目路径（请确保路径与您自己的目录一致）
import os
PROJECT_PATH = "/content/drive/MyDrive/CS6120_project"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 安装依赖（如果未安装）
!pip install sentence-transformers
!pip install datasets

import json
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader

# 载入预训练模型
model = SentenceTransformer('all-mpnet-base-v2')

# 假设你的数据存放在 combined.json 中
# 格式: {"train": [sentence1, sentence2, ...], "val": [...], "test": [...]}
data_path = "/content/drive/MyDrive/CS6120_project/data/processed/msmarco.json"
with open(data_path, 'r') as f:
    combined_data = json.load(f)

# 使用训练文本来构造训练样本
# 这里我们采用无监督对比学习的方式，将同一句子复制两次构成一个正样本对
train_examples = []
for sentence in combined_data["train"]:
    # 构造 InputExample: texts 为长度为2的列表，标签可以忽略（默认 None）
    train_examples.append(InputExample(texts=[sentence, sentence]))

# 构造 DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)

# 使用 MultipleNegativesRankingLoss 作为对比学习损失，
# 在批次内其它样本作为负样本，适合无监督/弱监督的场景
train_loss = losses.MultipleNegativesRankingLoss(model)

# 可选：如果有验证数据，可定义评估器（例如基于相似度的评估）
# 这里假设用验证集的前100个句子进行评估
val_sentences = combined_data["val"][:100]
# 我们可以使用 EmbeddingSimilarityEvaluator 来监控模型在验证集上的表现
val_scores = [1.0] * len(val_sentences)
evaluator = evaluation.EmbeddingSimilarityEvaluator(val_sentences, val_sentences, val_scores)

In [ ]:
# 开始微调
import os
os.environ["WANDB_DISABLED"] = "true" # disable log tracing by Wandb
num_epochs = 1  # 根据需要调整
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    warmup_steps=100,
    output_path="/content/drive/MyDrive/CS6120_project/model/finetuned_all_mpnet",
    evaluation_steps=500  # 可选，每500步评估一次
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Pearson Cosine,Spearman Cosine
500,0.000000,No log,nan,nan
1000,0.000000,No log,nan,nan
1500,0.000000,No log,nan,nan


/usr/local/lib/python3.11/dist-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:206: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_pearson, _ = pearsonr(labels, scores)
/usr/local/lib/python3.11/dist-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:207: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_spearman, _ = spearmanr(labels, scores)
/usr/local/lib/python3.11/dist-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:206: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_pearson, _ = pearsonr(labels, scores)
/usr/local/lib/python3.11/dist-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:207: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_spearman, _ = spearmanr(labels, scores)
